# 02 Apply all project cards

In [1]:
import os
import sys
import yaml
import pickle
import glob
import copy
import re

import pandas as pd

# https://github.com/BayAreaMetro/network_wrangler/tree/generic_agency
from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

# https://github.com/BayAreaMetro/Lasso/tree/mtc_parameters
from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc
from lasso import util

In [2]:
# Reload all modules (except those excluded by %aimport) every time before executing the Python code typed.
%load_ext autoreload
%autoreload 2

# Load memory profiler
%load_ext memory_profiler

# Remote I/O and parameters

In [3]:
# input directories - use Box
TM2_REBUILD_BOX_DIR      = r"C:\Users\{}\Box\Modeling and Surveys\Development\Travel Model Two Development\Travel Model Two Network Rebuild".format(os.getenv('USERNAME'))
TM2_NETWORKS_DATA_DIR    = os.path.join(TM2_REBUILD_BOX_DIR,    "travel-model-two-networks", "data")
input_data_interim_dir   = os.path.join(TM2_NETWORKS_DATA_DIR,  "interim")
input_data_processed_dir = os.path.join(TM2_NETWORKS_DATA_DIR,  "processed", "version_12")
input_data_external_dir  = os.path.join(TM2_NETWORKS_DATA_DIR,  "external")

# Lasso
LASSO_DIR                = 'C:/Users/{}/Documents/GitHub/Lasso'.format(os.getenv('USERNAME'))

# location of the project cards in the travel-model-two-networks repo
project_card_dir         ='C:/Users/{}/Documents/GitHub/travel-model-two-networks/project_cards'.format(os.getenv('USERNAME'))

In [4]:
# output folders - use local, most likely
output_dir          = r"C:\Users\{}\Documents\scratch\tm2_network_building\processed\version_12".format(os.getenv('USERNAME'))
output_standard_dir = os.path.join(output_dir, 'network_standard')
output_cube_dir     = os.path.join(output_dir, 'network_cube')
output_emme_dir     = os.path.join(output_dir, 'network_emme')

os.makedirs(output_standard_dir, exist_ok=True)
os.makedirs(output_cube_dir,     exist_ok=True)
os.makedirs(output_emme_dir,     exist_ok=True)

In [5]:
parameters = Parameters(lasso_base_dir = LASSO_DIR)

2022-01-20 12:11:06, INFO: Lasso base directory set as: C:/Users/lzorn/Documents/GitHub/Lasso
2022-01-20 12:11:06, INFO: Lasso base directory set as: C:/Users/lzorn/Documents/GitHub/Lasso


In [6]:
parameters.zones

4756

# Read scenario pickle

In [7]:
%%time
%%memit
# Wall time: ~2min
version_01_pickle_file_name = os.path.join(input_data_processed_dir, 'working_scenario_01.pickle')
scenario = pickle.load(open(version_01_pickle_file_name, 'rb'))

peak memory: 7626.91 MiB, increment: 7401.82 MiB
Wall time: 1min 59s


In [8]:
scenario.transit_net.feed.stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21881 entries, 0 to 21880
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   stop_name            21881 non-null  object 
 1   stop_lat             21881 non-null  float64
 2   stop_lon             21881 non-null  float64
 3   zone_id              8233 non-null   object 
 4   agency_raw_name      21881 non-null  object 
 5   stop_code            15289 non-null  object 
 6   location_type        5475 non-null   float64
 7   parent_station       249 non-null    object 
 8   stop_desc            409 non-null    object 
 9   stop_url             261 non-null    object 
 10  stop_timezone        126 non-null    object 
 11  wheelchair_boarding  377 non-null    float64
 12  platform_code        52 non-null     object 
 13  position             0 non-null      object 
 14  direction            0 non-null      object 
 15  * used by routes     124 non-null   

In [9]:
scenario.transit_net.feed.stops[
    scenario.transit_net.feed.stops.stop_name == "San Jose"]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
5466,San Jose,37.329935,-121.902496,56146,Capitol_2017_3_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6680,NaN,NaN,2192825


# Create scenario and apply projects

In [10]:
# this is required because the shape_foreign_key, which is 'id', is also expected in the links_df
# See https://app.asana.com/0/0/1201662144568397/f
scenario.road_net.links_df["id"] = scenario.road_net.links_df["shstGeometryId"]

In [11]:
%%time
%%memit

# todo: why not validate?  Requested some documentation on this here: 
# https://github.com/wsp-sag/network_wrangler/issues/278
scenario.remove_all_projects()
scenario.add_project_cards_from_tags(
    folder=project_card_dir,
    tags=['highway review', "Major Arterial Review", "Reversible Lanes", 'Bus Only', 'Toll Plaza'],
    validate=False
)

peak memory: 6604.23 MiB, increment: 0.01 MiB
Wall time: 7.24 s


In [12]:
%%time
%%memit
# Wall time: ~9min
scenario.apply_all_projects()

2022-01-20 12:13:19, INFO: Applying Year 2015 Modify Service Road Dumbarton Bridge Bypass
2022-01-20 12:13:19, INFO: Applying Year 2015 Modify Service Road Dumbarton Bridge Bypass
2022-01-20 12:13:19, INFO: Applying Project to Roadway Network: Year 2015 Modify Service Road Dumbarton Bridge Bypass


C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2022-01-20 12:13:20, INFO: Applying Year 2015 Modify Service Road Carquinez Bridge Bypass
2022-01-20 12:13:20, INFO: Applying Year 2015 Modify Service Road Carquinez Bridge Bypass
2022-01-20 12:13:20, INFO: Applying Project to Roadway Network: Year 2015 Modify Service Road Carquinez Bridge Bypass
2022-01-20 12:13:20, INFO: Applying add san mateo bridge toll plaza
2022-01-20 12:13:20, INFO: Applying add san mateo bridge toll plaza
2022-01-20 12:13:20, INFO: Applying Project to Roadway Network: add san mateo bridge toll plaza
2022-01-20 12:13:22, INFO: Applying Project to Transit Network: add san mateo bridge toll plaza
2022-01-20 12:13:22, WARNING: Roadway Deletion not yet implemented in Transit; ignoring
2022-01-20 12:13:22, INFO: Applying add san mateo bridge toll plaza
2022-01-20 12:13:22, INFO: Applying Project to Roadway Network: add san mateo bridge toll plaza
2022-01-20 12:16:16, INFO: Applying Year 2015 Correct As Is
2022-01-20 12:16:16, INFO: Applying Year 2015 Correct As Is
20

C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2022-01-20 12:16:17, INFO: Applying Year 2015 Modify Facility Type to Ramp shape incorrect
2022-01-20 12:16:17, INFO: Applying Year 2015 Modify Facility Type to Ramp shape incorrect
2022-01-20 12:16:17, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Ramp shape incorrect
2022-01-20 12:16:17, INFO: Applying Year 2015 Modify Facility Type to Ramp
2022-01-20 12:16:17, INFO: Applying Year 2015 Modify Facility Type to Ramp
2022-01-20 12:16:17, INFO: Applying Project to Roadway Network: Year 2015 Modify Facility Type to Ramp
2022-01-20 12:16:17, INFO: Applying Year 2015 Modify Lanes to 2 Lane and Facility Type to Ramp shape incorrect
2022-01-20 12:16:17, INFO: Applying Year 2015 Modify Lanes to 2 Lane and Facility Type to Ramp shape incorrect
2022-01-20 12:16:17, INFO: Applying Project to Roadway Network: Year 2015 Modify Lanes to 2 Lane and Facility Type to Ramp shape incorrect
2022-01-20 12:16:17, INFO: Applying Year 2015 Modify Lanes to 2 Lane and Facility Typ

2022-01-20 12:16:23, INFO: Applying Year 2015 Should be 3 Lane should be Undivided Arterial
2022-01-20 12:16:23, INFO: Applying Year 2015 Should be 3 Lane should be Undivided Arterial
2022-01-20 12:16:23, INFO: Applying Project to Roadway Network: Year 2015 Should be 3 Lane should be Undivided Arterial
2022-01-20 12:16:23, INFO: Applying Year 2015 Should be 2 Lane should be Undivided Arterial
2022-01-20 12:16:23, INFO: Applying Year 2015 Should be 2 Lane should be Undivided Arterial
2022-01-20 12:16:23, INFO: Applying Project to Roadway Network: Year 2015 Should be 2 Lane should be Undivided Arterial
2022-01-20 12:16:23, INFO: Applying Year 2015 Lane correct should be Undivided Arterial
2022-01-20 12:16:23, INFO: Applying Year 2015 Lane correct should be Undivided Arterial
2022-01-20 12:16:23, INFO: Applying Project to Roadway Network: Year 2015 Lane correct should be Undivided Arterial
2022-01-20 12:16:24, INFO: Applying Year 2015 Lanes Correct should be Expressway
2022-01-20 12:16:24

C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2022-01-20 12:21:54, INFO: Applying Year 2015 Bus Only Segments for Golden Gate Transit on US 101
2022-01-20 12:21:54, INFO: Applying Year 2015 Bus Only Segments for Golden Gate Transit on US 101
2022-01-20 12:21:54, INFO: Applying Project to Roadway Network: Year 2015 Bus Only Segments for Golden Gate Transit on US 101
2022-01-20 12:21:55, INFO: Applying year 2015 sterling on ramp HOV only in PM
2022-01-20 12:21:55, INFO: Applying Project to Roadway Network: year 2015 sterling on ramp HOV only in PM
2022-01-20 12:21:55, INFO: Applying year 2015 golden gate reversible lanes
2022-01-20 12:21:55, INFO: Applying Project to Roadway Network: year 2015 golden gate reversible lanes
2022-01-20 12:21:56, INFO: Applying year 2015 golden gate bridge shape correction
2022-01-20 12:21:56, INFO: Applying Project to Roadway Network: year 2015 golden gate bridge shape correction
peak memory: 8472.32 MiB, increment: 1868.09 MiB
Wall time: 8min 40s


In [13]:
# nice idea but this crashes
# step1_scenario.scenario_summary()

In [14]:
scenario.road_net.links_df[scenario.road_net.links_df.model_link_id == 8063198][["A", "B", "geometry", "id"]]

,A,B,geometry,id
1634794,5013515,5030001,"LINESTRING (-122.48221 37.83307, -122.48175 37...",d01b27cabff869e0bf4f7187ee77d6de


In [15]:
scenario.road_net.nodes_df[scenario.road_net.nodes_df.model_node_id == 5030001]

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry,farezone
644481,NaN,,Marin,1,0,0,5030001,0,-122.47658,37.808073,POINT (-122.47658 37.80807),


In [16]:
list(scenario.road_net.links_df[scenario.road_net.links_df.model_link_id == 8063198].geometry.iloc[0].coords)

[(-122.4822082, 37.8330742),
 (-122.48175233602524, 37.832883609672905),
 (-122.48116359114647, 37.83254572138074),
 (-122.4807545542717, 37.8322078315411),
 (-122.48029455542564, 37.83168881278591),
 (-122.48009875416754, 37.83132443825398),
 (-122.47998476028442, 37.83103844535508),
 (-122.47989490628242, 37.83066029748685),
 (-122.47976046055554, 37.829561062531596),
 (-122.47972592711449, 37.82925917389109),
 (-122.47969474643469, 37.82899541754155),
 (-122.47965652495623, 37.82866942822046),
 (-122.47961830347776, 37.828343967096075),
 (-122.4795787408948, 37.82800393945721),
 (-122.47953817248343, 37.82765676008782),
 (-122.47949860990049, 37.827314081061175),
 (-122.47865572571753, 37.820096795765785),
 (-122.4780549108982, 37.81498611149333),
 (-122.47746214270592, 37.80990050232683),
 (-122.47741788625717, 37.80948516210632),
 (-122.47734010219573, 37.80920968005912),
 (-122.47717916965483, 37.80883671811151),
 (-122.47690826654433, 37.808453158689176),
 (-122.47657969594002, 

In [17]:
list(scenario.road_net.shapes_df[scenario.road_net.shapes_df.id == "d01b27cabff869e0bf4f7187ee77d6de"].geometry.iloc[0].coords)

[(-122.4822082, 37.8330742),
 (-122.48175233602524, 37.832883609672905),
 (-122.48116359114647, 37.83254572138074),
 (-122.4807545542717, 37.8322078315411),
 (-122.48029455542564, 37.83168881278591),
 (-122.48009875416754, 37.83132443825398),
 (-122.47998476028442, 37.83103844535508),
 (-122.47989490628242, 37.83066029748685),
 (-122.47976046055554, 37.829561062531596),
 (-122.47972592711449, 37.82925917389109),
 (-122.47969474643469, 37.82899541754155),
 (-122.47965652495623, 37.82866942822046),
 (-122.47961830347776, 37.828343967096075),
 (-122.4795787408948, 37.82800393945721),
 (-122.47953817248343, 37.82765676008782),
 (-122.47949860990049, 37.827314081061175),
 (-122.47865572571753, 37.820096795765785),
 (-122.4780549108982, 37.81498611149333),
 (-122.47746214270592, 37.80990050232683),
 (-122.47741788625717, 37.80948516210632),
 (-122.47734010219573, 37.80920968005912),
 (-122.47717916965483, 37.80883671811151),
 (-122.47690826654433, 37.808453158689176),
 (-122.47657969594002, 

In [18]:
%%time
%%memit

scenario.remove_all_projects() # reset
scenario.add_project_cards_from_tags(
    folder=project_card_dir,
    tags=['Managed Lanes', 'toll review', 'Exclude Trucks'],
    validate=False
)

peak memory: 7088.15 MiB, increment: 0.07 MiB
Wall time: 8.21 s


In [19]:
%%time
%%memit
# Wall time: ~6min
scenario.apply_all_projects()

2022-01-20 12:22:12, ERROR: Projects year 2015 golden gate bridge shape correction has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 12:22:12, ERROR: Projects year 2015 golden gate reversible lanes has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 12:22:12, ERROR: Projects year 2015 modify toll links on golden gate bridge toll plaza on us101 sb - bridge pass has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 12:22:12, ERROR: Projects year 2015 modify hov lanes and use on san mateo bridge toll plaza toll plaza on ca-92 wb - bridge pass has add san mateo bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 12:22:12, INFO: Applying year 2015 modify hov lanes and use on san mateo bridge toll plaza toll plaza on ca-92 wb - bridge pass
2022-01-20 12:22:12, INFO: Applying year 2015 modify hov

C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2022-01-20 12:22:12, INFO: Applying Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2022-01-20 12:22:12, INFO: Applying Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2022-01-20 12:22:12, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Benicia Bridge Toll Plaza - Segment 03 - Bridge Pass
2022-01-20 12:22:14, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2022-01-20 12:22:14, INFO: Applying Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2022-01-20 12:22:14, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on Richmond-San Rafael Bridge Toll Plaza - Bridge Pass
2022-01-20 12:22:14, INFO: Applying Year 2015 Modify USE and HOV Lanes on US-101 SB (Segment 02) from South of Healdsburg to North of Petaluma
2022-01-20 12:22:14, INFO: Applying Year 2

2022-01-20 12:25:00, INFO: Applying Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2022-01-20 12:25:00, INFO: Applying Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2022-01-20 12:25:00, INFO: Applying Project to Roadway Network: Year 2015 Modify Express Lanes and USE on I-880 NB CA-237 to Dixon Road
2022-01-20 12:25:00, INFO: Applying Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2022-01-20 12:25:00, INFO: Applying Project to Roadway Network: Year 2015 Modify USE on I-80 WB to Bay Bridge Toll Plaza
2022-01-20 12:25:02, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 WB from Fairfield to I-680
2022-01-20 12:25:02, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 WB from Fairfield to I-680
2022-01-20 12:25:02, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on I-80 WB from Fairfield to I-680


C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2022-01-20 12:25:06, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 WB from Carquinez Bridge to Bay Bridge Toll Plaza
2022-01-20 12:25:06, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 WB from Carquinez Bridge to Bay Bridge Toll Plaza
2022-01-20 12:25:06, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on I-80 WB from Carquinez Bridge to Bay Bridge Toll Plaza
2022-01-20 12:25:14, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2022-01-20 12:25:14, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2022-01-20 12:25:14, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on I-80 EB from I-680 to Fairfield
2022-01-20 12:25:18, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from Bay Bridge to Carquinez Bridge Toll Plaza
2022-01-20 12:25:18, INFO: Applying Year 2015 Modify HOV Lanes and USE on I-80 EB from Bay Bridge to Carquinez

2022-01-20 12:26:14, INFO: Applying Year 2015 Modify HOV Lanes and USE on E Capitol Expressway NB US101 to I-680
2022-01-20 12:26:14, INFO: Applying Year 2015 Modify HOV Lanes and USE on E Capitol Expressway NB US101 to I-680
2022-01-20 12:26:14, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on E Capitol Expressway NB US101 to I-680
2022-01-20 12:26:25, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-92 WB Eden Landing Rd to San Mateo-Hayward Bridge Toll Plaza
2022-01-20 12:26:25, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-92 WB Eden Landing Rd to San Mateo-Hayward Bridge Toll Plaza
2022-01-20 12:26:25, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes and USE on CA-92 WB Eden Landing Rd to San Mateo-Hayward Bridge Toll Plaza
2022-01-20 12:26:27, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-87 SB US 101 to CA-85
2022-01-20 12:26:27, INFO: Applying Year 2015 Modify HOV Lanes and USE on CA-87 SB US 101 to

2022-01-20 12:27:08, INFO: Applying Project to Roadway Network: Year 2015 Exclude Trucks on I580
2022-01-20 12:27:08, INFO: Applying Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on Antioch Bridge NB on Route 160 - Bridge Pass
2022-01-20 12:27:08, INFO: Applying Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on Antioch Bridge NB on Route 160 - Bridge Pass
2022-01-20 12:27:08, INFO: Applying Project to Roadway Network: Year 2015 Modify HOV Lanes-Fastrak Lanes and USE on Antioch Bridge NB on Route 160 - Bridge Pass
peak memory: 8100.13 MiB, increment: 1011.98 MiB
Wall time: 5min


In [20]:
# todo: these QA queries would be more useful with notes about expected output
scenario.road_net.links_df.shape

(1634796, 44)

In [21]:
scenario.road_net.links_df.model_link_id.nunique()

1634796

In [22]:
scenario.road_net.links_df[scenario.road_net.links_df.model_link_id == 3149471].useclass

762806    {'default': 0, 'timeofday': [{'time': (21600, ...
Name: useclass, dtype: object

In [23]:
scenario.road_net.links_df.county.isnull().sum()

0

In [24]:
scenario.road_net.links_df.model_link_id.max()

8063199

In [25]:
%%time
%%memit

scenario.remove_all_projects() # reset
scenario.add_project_cards_from_tags(
    folder=project_card_dir,
    tags=['Major Transit links'],
    validate=False
)

peak memory: 7201.49 MiB, increment: 0.55 MiB
Wall time: 7.65 s


In [26]:
%%time
%%memit
# Wall time: ~45min
scenario.apply_all_projects()

2022-01-20 12:27:22, ERROR: Projects year 2015 golden gate bridge shape correction has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 12:27:22, ERROR: Projects year 2015 golden gate reversible lanes has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 12:27:22, ERROR: Projects year 2015 modify toll links on golden gate bridge toll plaza on us101 sb - bridge pass has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 12:27:22, ERROR: Projects year 2015 modify hov lanes and use on san mateo bridge toll plaza toll plaza on ca-92 wb - bridge pass has add san mateo bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 12:27:22, INFO: Applying move street rail to street connector links
2022-01-20 12:27:22, INFO: Applying move street rail to street connector links
2022-01-20 12:27:22, INFO: Applying P

C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\geopandas\geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


2022-01-20 12:29:48, INFO: Applying move street rail to street connector links
2022-01-20 12:29:48, INFO: Applying Project to Roadway Network: move street rail to street connector links
2022-01-20 12:32:18, INFO: Applying move street rail to street connector links
2022-01-20 12:32:18, INFO: Applying Project to Roadway Network: move street rail to street connector links
2022-01-20 12:34:52, INFO: Applying move street rail to street connector links
2022-01-20 12:34:52, INFO: Applying Project to Roadway Network: move street rail to street connector links
2022-01-20 12:37:27, INFO: Applying move street rail to street connector links
2022-01-20 12:37:27, INFO: Applying Project to Roadway Network: move street rail to street connector links
2022-01-20 12:40:01, INFO: Applying move street rail to street connector links
2022-01-20 12:40:01, INFO: Applying Project to Roadway Network: move street rail to street connector links
2022-01-20 12:42:39, INFO: Applying move street rail to street connect

In [27]:
scenario.road_net.links_df.shape

(1634814, 44)

In [28]:
scenario.road_net.links_df.columns

Index(['access', 'bike_access', 'drive_access', 'fromIntersectionId', 'lanes',
       'maxspeed', 'name', 'oneWay', 'ref', 'roadway', 'shstGeometryId',
       'shstReferenceId', 'toIntersectionId', 'u', 'v', 'walk_access', 'wayId',
       'county', 'model_link_id', 'A', 'B', 'rail_traveltime', 'rail_only',
       'locationReferences', 'geometry', 'ft_cal', 'ft', 'useclass',
       'assignable_cal', 'assignable', 'transit', 'id', 'tollbooth',
       'bus_only', 'transit_access', 'managed', 'ML_lanes', 'segment_id',
       'ML_tollbooth', 'ML_useclass', 'ML_access', 'ML_egress', 'ML_tollseg',
       'tollseg'],
      dtype='object')

In [29]:
print(scenario.scenario_summary())

2022-01-20 13:13:00, INFO: Summarizing Scenario
------------------------------
Scenario created on 2022-01-20 13:13:00.922562
Base Scenario:
--Road Network:
----Link File: C:\Users\lzorn\Box\Modeling and Surveys\Development\Travel Model Two Development\Travel Model Two Network Rebuild\travel-model-two-networks\data\interim\step8_standard_format\link.json
----Node File: C:\Users\lzorn\Box\Modeling and Surveys\Development\Travel Model Two Development\Travel Model Two Network Rebuild\travel-model-two-networks\data\interim\step8_standard_format\node.geojson
----Shape File: C:\Users\lzorn\Box\Modeling and Surveys\Development\Travel Model Two Development\Travel Model Two Network Rebuild\travel-model-two-networks\data\interim\step8_standard_format\shape.geojson
--Transit Network:
----Feed Path: C:\Users\lzorn\Box\Modeling and Surveys\Development\Travel Model Two Development\Travel Model Two Network Rebuild\travel-model-two-networks\data\processed\version_12\standard_transit_pre_base_project_c

In [30]:
%%time
%%memit

# why is this necessary?  why not validate consistency?
scenario.transit_net.set_roadnet(scenario.road_net, 
                                 validate_consistency = False)

scenario.remove_all_projects() # reset
scenario.add_project_cards_from_tags(
    folder=project_card_dir,
    tags=['Minor Transit', 'Add Transit', 'Major Transit', 'Toll Plaza Transit'],
    validate=False
)

peak memory: 13380.31 MiB, increment: 6063.34 MiB
Wall time: 6min 22s


In [31]:
%%time
%%memit
# Wall time: ~27min
scenario.apply_all_projects()

2022-01-20 13:19:31, ERROR: Projects year 2015 golden gate bridge shape correction has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 13:19:31, ERROR: Projects year 2015 golden gate reversible lanes has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 13:19:31, ERROR: Projects year 2015 modify toll links on golden gate bridge toll plaza on us101 sb - bridge pass has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 13:19:31, ERROR: Projects year 2015 modify hov lanes and use on san mateo bridge toll plaza toll plaza on ca-92 wb - bridge pass has add san mateo bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 13:19:31, ERROR: Projects sfmta route n-judah has move street rail to street connector links as prerequisite project which is missing for the scenario
2022-01-20 13:19:31, ERROR: Proje

C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with d

2022-01-20 13:19:56, INFO: Creating a new stop in stops.txt for node ID: 2078170
2022-01-20 13:19:56, INFO: Creating a new stop in stops.txt for node ID: 2104183
2022-01-20 13:19:57, INFO: Creating a new stop in stops.txt for node ID: 2015328


C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with d

2022-01-20 13:19:58, INFO: Applying VTA Route 22
2022-01-20 13:19:58, INFO: Applying Project to Transit Network: VTA Route 22
2022-01-20 13:19:59, WARNING: Missing connections from node 2033565 to node 2103889 for the new routing, find complete path using default graph
2022-01-20 13:19:59, INFO: Routing path from node 2033565 to node 2103889 for missing connections: ['2033565', '2155466', '2103889'].
2022-01-20 13:19:59, WARNING: Missing connections from node 2035353 to node 2158621 for the new routing, find complete path using default graph
2022-01-20 13:19:59, INFO: Routing path from node 2035353 to node 2158621 for missing connections: ['2035353', '2061243', '2158621'].
2022-01-20 13:19:59, WARNING: Missing connections from node 2026323 to node 2102903 for the new routing, find complete path using default graph
2022-01-20 13:19:59, INFO: Routing path from node 2026323 to node 2102903 for missing connections: ['2026323', '2045820', '2102903'].
2022-01-20 13:19:59, INFO: Creating a ne

...

2022-01-20 13:44:41, INFO: Applying year 2015 ac transit route 40 edits
2022-01-20 13:44:41, INFO: Applying year 2015 ac transit route 40 edits
2022-01-20 13:44:41, INFO: Applying Project to Transit Network: year 2015 ac transit route 40 edits
2022-01-20 13:44:44, INFO: Applying year 2015 ac transit route 38 edits
2022-01-20 13:44:44, INFO: Applying year 2015 ac transit route 38 edits
2022-01-20 13:44:44, INFO: Applying Project to Transit Network: year 2015 ac transit route 38 edits
2022-01-20 13:44:45, INFO: Creating a new stop in stops.txt for node ID: 2503531
2022-01-20 13:44:45, INFO: Creating a new stop in stops.txt for node ID: 2539453
2022-01-20 13:44:45, INFO: Creating a new stop in stops.txt for node ID: 2564896


C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with d

2022-01-20 13:44:47, INFO: Applying year 2015 ac transit route 06 edits
2022-01-20 13:44:47, INFO: Applying year 2015 ac transit route 06 edits
2022-01-20 13:44:47, INFO: Applying Project to Transit Network: year 2015 ac transit route 06 edits
2022-01-20 13:44:48, INFO: Creating a new stop in stops.txt for node ID: 2592196
2022-01-20 13:44:48, INFO: Creating a new stop in stops.txt for node ID: 2606436
2022-01-20 13:44:48, INFO: Creating a new stop in stops.txt for node ID: 2519741
2022-01-20 13:44:48, INFO: Creating a new stop in stops.txt for node ID: 2571302
2022-01-20 13:44:48, INFO: Creating a new stop in stops.txt for node ID: 2576003
2022-01-20 13:44:48, INFO: Creating a new stop in stops.txt for node ID: 2521855


C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\numpy\core\fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with d

2022-01-20 13:44:49, INFO: Applying ac transit fare system change to zonal fare
2022-01-20 13:44:49, INFO: Applying Project to Transit Network: ac transit fare system change to zonal fare
peak memory: 12969.22 MiB, increment: 1400.92 MiB
Wall time: 25min 25s


In [32]:
scenario.road_net = mtc.add_centroid_and_centroid_connector(
    roadway_network = scenario.road_net,
    parameters = parameters)

2022-01-20 13:44:50, INFO: Adding centroid and centroid connector to standard network
2022-01-20 13:44:50, INFO: Lasso base directory set as: C:/Users/lzorn/Documents/GitHub/Lasso
2022-01-20 13:44:50, INFO: Lasso base directory set as: C:/Users/lzorn/Documents/GitHub/Lasso
2022-01-20 13:45:28, INFO: Finished adding centroid and centroid connectors


In [33]:
print(scenario.road_net.links_df.shape)
print(scenario.road_net.links_df.model_link_id.nunique())
print(scenario.road_net.nodes_df.shape)
print(scenario.road_net.nodes_df.model_node_id.nunique())
print(scenario.road_net.shapes_df.shape)

(2199316, 44)
2199316
(688964, 12)
688964
(1152265, 6)


In [34]:
scenario.road_net.links_df.locationReferences.iloc[-1]

[{'sequence': 1,
  'point': [-122.50580293859502, 37.94830473568058],
  'distanceToNextRef': 51.382886808196375,
  'bearing': 0,
  'intersectionId': nan},
 {'sequence': 2,
  'point': [-122.50576480000001, 37.9487303],
  'intersectionId': '44c53879b5a9797cbb2eff72b5f64699'}]

In [35]:
%%time
%%memit
# Wall time: ~6min
scenario.remove_all_projects() # reset
scenario.add_project_cards_from_tags(
    folder=project_card_dir,
    tags=['External Stations Review'],
    validate=False
)

peak memory: 12466.98 MiB, increment: 1.75 MiB
Wall time: 12.4 s


In [36]:
%%time
%%memit
# Wall time: ~1h 50min
scenario.apply_all_projects()

2022-01-20 13:45:50, ERROR: Projects year 2015 golden gate bridge shape correction has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 13:45:50, ERROR: Projects year 2015 golden gate reversible lanes has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 13:45:50, ERROR: Projects year 2015 modify toll links on golden gate bridge toll plaza on us101 sb - bridge pass has add golden gate bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 13:45:50, ERROR: Projects year 2015 modify hov lanes and use on san mateo bridge toll plaza toll plaza on ca-92 wb - bridge pass has add san mateo bridge toll plaza as prerequisite project which is missing for the scenario
2022-01-20 13:45:50, ERROR: Projects ac transit fare system change to zonal fare has year 2015 ac transit route 06 edits as prerequisite project which is missing for the scenario
2022-01-20 13:45

2022-01-20 14:30:32, INFO: Applying Year 2015 Add centroid connector at External Station connecting Pacheco Pass Highway CA-156 NB
2022-01-20 14:30:32, INFO: Applying Year 2015 Add centroid connector at External Station connecting Pacheco Pass Highway CA-156 NB
2022-01-20 14:30:32, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Pacheco Pass Highway CA-156 NB
2022-01-20 14:33:53, INFO: Applying Year 2015 Add centroid connector at External Station connecting Pacheco Pass Highway CA-152 SB
2022-01-20 14:33:53, INFO: Applying Year 2015 Add centroid connector at External Station connecting Pacheco Pass Highway CA-152 SB
2022-01-20 14:33:53, INFO: Applying Project to Roadway Network: Year 2015 Add centroid connector at External Station connecting Pacheco Pass Highway CA-152 SB
2022-01-20 14:37:20, INFO: Applying Year 2015 Add centroid connector at External Station connecting Pacheco Pass Highway CA-152 NB
2022-01-20 14:37:20, INFO: 

In [37]:
print(scenario.road_net.links_df.shape)
print(scenario.road_net.links_df.model_link_id.nunique())
print(scenario.road_net.nodes_df.shape)
print(scenario.road_net.nodes_df.model_node_id.nunique())

(2199342, 44)
2199342
(688964, 12)
688964


In [38]:
# add taps and tap connectors
scenario.road_net = mtc.add_tap_and_tap_connector(
    roadway_network = scenario.road_net,
    transit_network = scenario.transit_net, 
    parameters = parameters)

2022-01-20 15:15:51, INFO: Adding tap and tap connector to standard network
2022-01-20 15:15:51, INFO: Lasso base directory set as: C:/Users/lzorn/Documents/GitHub/Lasso
2022-01-20 15:15:51, INFO: Lasso base directory set as: C:/Users/lzorn/Documents/GitHub/Lasso
2022-01-20 15:15:52, INFO: There are 601 stops not connected to taps, connecting them to the closest taps.
snapping gtfs stops to roadway node osmid...


c:\users\lzorn\documents\github\lasso\lasso\mtc.py:2636: FutureWarning: CRS mismatch between CRS of the passed geometries and 'crs'. Use 'GeoDataFrame.set_crs(crs, allow_override=True)' to overwrite CRS or 'GeoDataFrame.to_crs(crs)' to reproject geometries. CRS mismatch will raise an error in the future versions of GeoPandas.
  crs = "EPSG:4326"


2022-01-20 15:16:04, INFO: Finished adding tap and tap connectors


In [39]:
%%time
%memit
# add county
scenario.road_net = mtc.calculate_county(
    roadway_network = scenario.road_net,
    parameters = parameters,
    network_variable = 'county'
)

2022-01-20 15:16:04, INFO: County Variable 'county' already in network, calculating for missing ones
2022-01-20 15:16:04, INFO: Adding roadway network variable for county using a spatial join with: C:/Users/lzorn/Documents/GitHub/Lasso\mtc_data\county\county.shp


c:\users\lzorn\documents\github\lasso\lasso\mtc.py:2803: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  link_centroids_gdf["geometry"] = link_centroids_gdf["geometry"].centroid


2022-01-20 15:22:11, INFO: Finished Calculating link county variable: county
2022-01-20 15:23:38, INFO: Finished Calculating node county variable: county


In [40]:
scenario.road_net.links_df.county.value_counts()

Santa Clara      660008
Alameda          443532
Contra Costa     326621
San Mateo        203882
Sonoma           178792
Solano           157454
San Francisco    133512
Marin             84883
Napa              47068
External             28
Name: county, dtype: int64

In [41]:
scenario.road_net.nodes_df.county.value_counts()

Santa Clara      203748
Alameda          137128
Contra Costa     104665
San Mateo         61887
Sonoma            59970
Solano            50710
San Francisco     33406
Marin             28439
Napa              14990
External             21
Name: county, dtype: int64

# sanity check

In [42]:
scenario.road_net.links_df[scenario.road_net.links_df.model_link_id == 3150085] # should be ramp lanes 2

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
765902,,False,True,d1b76ff92ba7a2915de8efc97c782d73,2,,,True,,motorway_link,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
scenario.road_net.links_df[scenario.road_net.links_df.model_link_id == 3107408] # bay bridge toll link

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
548523,"['nan', 'nan']",False,True,826a99158fd58e5483d1085c9e0d65b3,"{'default': 4, 'timeofday': [{'time': (21600, ...","['nan', 'nan']","['nan', 'nan']","[True, True]","['I 80', 'I 80']",motorway,...,NaN,1,"{'default': 0, 'timeofday': [{'time': (21600, ...",5,5,"{'default': 0, 'timeofday': [{'time': (21600, ...",all,all,NaN,NaN


In [44]:
scenario.road_net.links_df[
    scenario.road_net.links_df.model_link_id == 26067343
].iloc[0][["A", "B"]]

A     900020
B    2192724
Name: 2199322, dtype: object

In [45]:
# this link is deleted and splitted into two for toll plaza at GGB
scenario.road_net.links_df[
    scenario.road_net.links_df.model_link_id == 8016300]

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg


In [46]:
# golden gate bridge
scenario.road_net.links_df[
    scenario.road_net.links_df.model_link_id == 8063198].lanes.iloc[0]

{'default': 3,
 'timeofday': [{'time': (21600, 36000), 'value': 4},
  {'time': (54000, 68400), 'value': 2}]}

In [47]:
# golden gate bridge HOV
scenario.road_net.links_df[
    scenario.road_net.links_df.model_link_id == 8063199].ML_lanes.iloc[0]

{'default': 0,
 'timeofday': [{'time': (21600, 36000), 'value': 2},
  {'time': (54000, 68400), 'value': 2}]}

In [48]:
# check if new node is created for toll plaza at GGB
scenario.road_net.nodes_df[scenario.road_net.nodes_df.model_node_id == 5030001]

,osm_node_id,shst_node_id,county,drive_access,walk_access,bike_access,model_node_id,rail_only,X,Y,geometry,farezone
644481,NaN,,Marin,1.0,0.0,0.0,5030001,0.0,-122.47658,37.808073,POINT (-122.47658 37.80807),


In [49]:
# golden gate bridge toll plaza
scenario.road_net.links_df[
    scenario.road_net.links_df.model_link_id == 8063199].lanes.iloc[0]

{'default': 9,
 'timeofday': [{'time': (21600, 36000), 'value': 7},
  {'time': (54000, 68400), 'value': 7}]}

In [50]:
# check shapes for new links
# example: toll plaza at GGB and a new rail-to-street link
scenario.road_net.links_df[
    scenario.road_net.links_df.model_link_id.isin([8063198, 8063199, 26067378])][["id", "geometry"]]

,id,geometry
1634794,d01b27cabff869e0bf4f7187ee77d6de,"LINESTRING (-122.48221 37.83307, -122.48175 37..."
1634795,b46d3b9a4683bfe26b97e2a66181d8dd,"LINESTRING (-122.47658 37.80807, -122.47536 37..."
1634796,ba0e166e3a9f6c1cc8d7e331415769b2,"LINESTRING (-122.42941 37.76947, -122.43026 37..."


In [51]:
scenario.road_net.shapes_df[
    scenario.road_net.shapes_df.id.isin(["b46d3b9a4683bfe26b97e2a66181d8dd", "ba0e166e3a9f6c1cc8d7e331415769b2"])][
        ["id", "geometry"]]

,id,geometry
869991,b46d3b9a4683bfe26b97e2a66181d8dd,"LINESTRING (-122.47658 37.80807, -122.47536 37..."
869992,ba0e166e3a9f6c1cc8d7e331415769b2,"LINESTRING (-122.42941 37.76947, -122.43026 37..."


In [52]:
scenario.transit_net.feed.shapes.shape_id.nunique()

1592

In [53]:
scenario.transit_net.feed.shapes[
    scenario.transit_net.feed.shapes.shape_id.isin(
        ["1169", "1168", "939", "940", "943", "944"]
    )
].groupby(["shape_id"]).count()

,shape_pt_sequence,shape_osm_node_id,shape_shst_node_id,shape_model_node_id,shape_pt_lat,shape_pt_lon
shape_id,,,,,,
1168,177,108,108,177,0,0
1169,176,65,65,176,0,0
939,26,22,22,26,0,0
940,161,157,157,161,0,0
943,158,154,154,158,0,0
944,130,128,128,130,0,0


In [54]:
scenario.road_net.links_df[
    scenario.road_net.links_df.A.isin([2579949, 2607569])][["A", "B", "roadway","assignable"]]

,A,B,roadway,assignable
514595,2579949,2535246,residential,0
842601,2579949,2604856,residential,1
846406,2607569,2586861,residential,1
1009927,2607569,2546185,residential,1
1077889,2607569,2541364,residential,0
1645033,2579949,300963,taz,1
1645951,2607569,300963,taz,1
1820125,2579949,318103,maz,1
1820126,2579949,327248,maz,1
1820127,2579949,328044,maz,1


In [55]:
# check Service Road Dumbarton Bridge Bypass is not assignable
scenario.road_net.links_df[
    scenario.road_net.links_df.model_link_id == 3152465][["A", "B", "roadway","assignable"]]

,A,B,roadway,assignable
778002,2552715,2540313,motorway_link,0


In [56]:
# check should be 3 lanes
scenario.road_net.links_df[
    scenario.road_net.links_df.model_link_id == 2217343][["A", "B", "roadway","assignable", "lanes"]]

,A,B,roadway,assignable,lanes
690625,2153360,2071468,motorway,1,"{'default': 3, 'timeofday': [{'time': (21600, ..."


In [57]:
# should be ramps
scenario.road_net.links_df[
    scenario.road_net.links_df.model_link_id.isin(
        [3031886,3006561,3046803,3101610,3011518,3024387,4018032])][["A", "B", "roadway","assignable", "ft", "walk_access"]]

,A,B,roadway,assignable,ft,walk_access
33757,2506600,2542798,motorway_link,1,3,1
59112,2511462,2523557,motorway_link,1,3,1
123885,3017507,3058438,motorway_link,1,3,1
124668,2523557,2549665,motorway_link,1,3,1
162620,2501507,2506600,motorway_link,1,3,1
238525,2542798,2518014,motorway_link,1,3,1
518512,2580420,2511462,motorway_link,1,3,1


In [58]:
# zone id should be "2" (ac transit), "1" (dumbarton union city), "2" (ac transit dumbarton), "1" (samtrans) , "1" (samtrans)
scenario.transit_net.feed.stops[scenario.transit_net.feed.stops.stop_id.isin(
    ["101523872", "4694", "4669", "102036044", "4187"])]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
4130,Transbay Temp Terminal,37.789707,-122.393626,2,ACTransit_2015_8_14,50751,NaN,place_ETSF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4187,3955000020,459ceb872e72e49fd4041a78884935e2,1021597
4611,Union City BART,37.590360,-122.017465,1,ACTransit_2015_8_14,50781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4669,1025879893,84c3f377f9f0065bf14be3af92202d0d,2509987
4636,Stanford Oval,37.429272,-122.168925,2,ACTransit_2015_8_14,55432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4694,522171022,6d2aeffcbad07f0873a268bea42838f3,2149155
22309,NaN,37.444794,-122.168849,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102036044,NaN,NaN,2036044
22338,NaN,37.673730,-122.467029,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101523872,NaN,NaN,1523872


In [59]:
# check zone_id is removed for gg ferry stops
scenario.transit_net.feed.stops[scenario.transit_net.feed.stops.stop_id.isin(
    ["7706", "7707", "7708"])]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
7603,San Franciso Ferry Terminal,37.796169,-122.391617,NaN,GGTransit_2015_9_3,43000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7706,NaN,NaN,1027623
7604,Sausalito Ferry Terminal,37.856403,-122.478249,NaN,GGTransit_2015_9_3,43003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7707,NaN,NaN,5026533
7605,Larkspur Ferry Terminal,37.945316,-122.508736,NaN,GGTransit_2015_9_3,43004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7708,NaN,NaN,5026532


In [60]:
# check zone_id is updated for fairfield
scenario.transit_net.feed.stops[scenario.transit_net.feed.stops.stop_id.isin(
    ["6817", "6972"])]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
6741,El Cerrito Del Norte BART,37.924894,-122.317048,671,Fairfield_2015_10_14,400,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6972,1781110705,e235e2f09ba74bdbe6b6abb3e44fc47e,3060051
6779,Main Street and Lotz Way,38.243081,-122.040478,663,Fairfield_2015_10_14,172,0.0,NaN,Far-Side,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6817,62481309,52b70c286cb2792528a436f5736b33f7,3512083


In [61]:
# check USECLASS on I-80 WB to Bay Bridge Toll Plaza
scenario.road_net.links_df[scenario.road_net.links_df['model_link_id'].isin([3061568, 3149471])].useclass

314471    {'default': 0, 'timeofday': [{'time': (21600, ...
762806    {'default': 0, 'timeofday': [{'time': (21600, ...
Name: useclass, dtype: object

In [62]:
# check zone-id for new stops on SamTrans
scenario.transit_net.feed.stops[
    scenario.transit_net.feed.stops.stop_id.astype(int).isin(
    [102036044,101511493,101516920,101525498,101516256,101513094,101513976,101528479,101529690,101529454,101519622,101513122,101514054,101543182,101512385,101534620,101503246,101537667,101538028,101526122,101519051,101503077,101508313,101508126,101537888,101519350,101516319,101509775,101536669])
]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
22309,NaN,37.444794,-122.168849,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102036044,NaN,NaN,2036044
22310,NaN,37.472389,-122.216100,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101511493,NaN,NaN,1511493
22311,NaN,37.483308,-122.231206,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101516920,NaN,NaN,1516920
22312,NaN,37.485404,-122.233938,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101525498,NaN,NaN,1525498
22313,NaN,37.489469,-122.238568,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101516256,NaN,NaN,1516256
22314,NaN,37.503227,-122.255333,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101513094,NaN,NaN,1513094
22315,NaN,37.507359,-122.260283,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101513976,NaN,NaN,1513976
22316,NaN,37.526752,-122.283581,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101528479,NaN,NaN,1528479
22317,NaN,37.540371,-122.301416,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101529690,NaN,NaN,1529690
22318,NaN,37.559804,-122.322379,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101529454,NaN,NaN,1529454


In [63]:
# check zone-id for new stops on SamTrans
scenario.transit_net.feed.stops[
    scenario.transit_net.feed.stops.stop_id.astype(int).isin(
    [101523872,101545076,101503345,101519909,101504028,101514714,101506746])
]

,stop_name,stop_lat,stop_lon,zone_id,agency_raw_name,stop_code,location_type,parent_station,stop_desc,stop_url,stop_timezone,wheelchair_boarding,platform_code,position,direction,* used by routes,stop_id,osm_node_id,shst_node_id,model_node_id
22338,NaN,37.673730,-122.467029,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101523872,NaN,NaN,1523872
22339,NaN,37.682409,-122.487089,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101545076,NaN,NaN,1545076
22340,NaN,37.697643,-122.483092,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101503345,NaN,NaN,1503345
22341,NaN,37.701361,-122.484757,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101519909,NaN,NaN,1519909
22342,NaN,37.702417,-122.481156,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101504028,NaN,NaN,1504028
22343,NaN,37.704069,-122.475537,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101514714,NaN,NaN,1514714
22344,NaN,37.671171,-122.464817,1,SamTrans_2015_8_20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101506746,NaN,NaN,1506746


In [64]:
# check San Mateo bridge toll plaza
scenario.road_net.links_df[
    scenario.road_net.links_df.model_link_id.isin(
    [3315463,3315462, 3024600])
].lanes.tolist()

[{'default': 4,
  'timeofday': [{'time': (21600, 36000), 'value': 3},
   {'time': (54000, 68400), 'value': 3}]},
 {'default': 4,
  'timeofday': [{'time': (21600, 36000), 'value': 3},
   {'time': (54000, 68400), 'value': 3}]},
 {'default': 7,
  'timeofday': [{'time': (21600, 36000), 'value': 5},
   {'time': (54000, 68400), 'value': 5}]}]

In [65]:
# check Bay Bridge west pan
scenario.road_net.links_df[
    scenario.road_net.links_df.model_link_id.isin(
    [38545])
]

,access,bike_access,drive_access,fromIntersectionId,lanes,maxspeed,name,oneWay,ref,roadway,...,transit_access,managed,ML_lanes,segment_id,ML_tollbooth,ML_useclass,ML_access,ML_egress,ML_tollseg,tollseg
818708,"['nan', '']",False,True,dc196b0fb1ac8053a741cd53df6ff64d,5,"['50 mph', '']","['San Francisco – Oakland Bay Bridge', '']","[True, True]","['I 80', '']",motorway,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Write the scenario as a pickle

In [66]:
%%time
%%memit
# Wall time: 3min
working_scenario_filename = os.path.join(output_standard_dir, 'working_scenario_v12.pickle')
pickle.dump(scenario, open(working_scenario_filename, 'wb'))

peak memory: 18340.12 MiB, increment: 5636.53 MiB
Wall time: 3min 5s


### Write the standard networks

In [98]:
%%time
%%memit
scenario.road_net.write(path = output_standard_dir, filename = "v12")
scenario.transit_net.write(path = output_standard_dir, filename = "v12")

# save out link feather
# to_feather() does not like mixed type
links_feather = scenario.road_net.links_df.copy()
links_feather.drop("geometry", axis = 1, inplace = True)

for c in links_feather.columns:
    if c in parameters.int_col:
        links_feather[c] = links_feather[c].astype(int)
    else:
        links_feather[c] = links_feather[c].astype(str)
links_feather.to_feather(path = os.path.join(output_standard_dir, "v12_links.feather"))

del links_feather

2022-01-21 10:04:24, INFO: Writing transit to directory: C:\Users\lzorn\Documents\scratch\tm2_network_building\processed\version_12\network_standard


C:\Users\lzorn\.conda\envs\tm2_network_dev\lib\site-packages\ipykernel_launcher.py:14: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  


peak memory: 20251.02 MiB, increment: 5111.56 MiB
Wall time: 11min 16s


## Make Travel Model Networks

The model network code has been removed from this notebook;
Please use [convert_scenario_to_modeled_networks.py](https://github.com/BayAreaMetro/travel-model-two-networks/blob/develop/src/scripts/convert_scenario_to_modeled_networks.py)